# Deep-subspace-clustering-networks
My GitHub：https://github.com/Elliot-Wang/Deep-subspace-clustering-networks.git  
My Account：15260223133@163.com

In [1]:
!pip install munkres==1.0.8
#!pip install tensorflow==1.14.0
%tensorflow_version 1.x
!git clone https://github.com/Elliot-Wang/DSC-rebuilt.git
!mv DSC-rebuilt/*.py ./

  Created wheel for munkres: filename=munkres-1.0.8-cp36-none-any.whl size=11236 sha256=900af597200ff40db6bdf8857b3f8e2c7a071186d678566a1806bfb1027f6f00
  Stored in directory: /root/.cache/pip/wheels/9f/61/27/66dfa0b5b370aca085ef8dfbe49b6b6d0763f26c3d82f51e5d
Successfully built munkres
Cloning into 'DSC-rebuilt'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 34 (delta 6), reused 34 (delta 6), pack-reused 0
Unpacking objects: 100% (34/34), done.


In [4]:
# Code Authors: Pan Ji,     University of Adelaide,         pan.ji@adelaide.edu.au
#               Tong Zhang, Australian National University, tong.zhang@anu.edu.au
# Copyright Reserved!
import tensorflow as tf
import numpy as np
from tensorflow.contrib import layers
import scipy.io as sio
from scipy.sparse.linalg import svds
from sklearn import cluster
from sklearn.preprocessing import normalize
from munkres import Munkres
import time

class ConvAE(object):
    def __init__(self, n_input, kernel_size, n_hidden, reg_constant1 = 1.0, re_constant2 = 1.0, batch_size = 200, reg = None, \
                denoise = False, model_path = None, restore_path = None, \
                logs_path = 'pretrain-model-EYaleB/logs'):
        self.n_input = n_input
        self.kernel_size = kernel_size
        self.n_hidden = n_hidden
        self.batch_size = batch_size
        self.reg = reg
        self.model_path = model_path
        self.restore_path = restore_path
        self.iter = 0
        
        #input required to be fed
        self.x = tf.placeholder(tf.float32, [None, n_input[0], n_input[1], 1])
        self.learning_rate = tf.placeholder(tf.float32, [])
        
        weights = self._initialize_weights()
        
        if denoise == False:
            x_input = self.x
            latent, shape = self.encoder(x_input, weights)
        else:
            x_input = tf.add(self.x, tf.random_normal(shape=tf.shape(self.x),
                                               mean = 0,
                                               stddev = 0.2,
                                               dtype=tf.float32))
            latent, shape = self.encoder(x_input, weights)
                   
        z = tf.reshape(latent, [batch_size, -1])  
        Coef = weights['Coef']         
        z_c = tf.matmul(Coef,z)    
        self.Coef = Coef        
        latent_c = tf.reshape(z_c, tf.shape(latent)) 
        self.z = z       
        
        self.x_r = self.decoder(latent_c, weights, shape)                
        
        # l_2 reconstruction loss 
        self.reconst_cost = 0.5 * tf.reduce_sum(tf.pow(tf.subtract(self.x_r, self.x), 2.0))
        tf.summary.scalar("recons_loss", self.reconst_cost)
                
        self.reg_losses = tf.reduce_sum(tf.pow(self.Coef,2.0))
        tf.summary.scalar("reg_loss", reg_constant1 * self.reg_losses )
        
        self.selfexpress_losses = 0.5 * tf.reduce_sum(tf.pow(tf.subtract(z_c, z), 2.0))
        tf.summary.scalar("selfexpress_loss", re_constant2 * self.selfexpress_losses )
        
        self.loss = self.reconst_cost + reg_constant1 * self.reg_losses + re_constant2 * self.selfexpress_losses  
        self.merged_summary_op = tf.summary.merge_all()

        self.optimizer = tf.train.AdamOptimizer(learning_rate = self.learning_rate).minimize(self.loss) #GradientDescentOptimizer #AdamOptimizer
        self.init = tf.global_variables_initializer()
        self.sess = tf.InteractiveSession()
        self.sess.run(self.init)        
        self.saver = tf.train.Saver([v for v in tf.trainable_variables() if not (v.name.startswith("Coef"))])              
        self.summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
        
    def _initialize_weights(self):
        all_weights = dict()
        all_weights['enc_w0'] = tf.get_variable("enc_w0", shape=[self.kernel_size[0], self.kernel_size[0], 1, self.n_hidden[0]],
            initializer=layers.xavier_initializer_conv2d(),regularizer = self.reg)
        all_weights['enc_b0'] = tf.Variable(tf.zeros([self.n_hidden[0]], dtype = tf.float32))

        all_weights['enc_w1'] = tf.get_variable("enc_w1", shape=[self.kernel_size[1], self.kernel_size[1], self.n_hidden[0],self.n_hidden[1]],
            initializer=layers.xavier_initializer_conv2d(),regularizer = self.reg)
        all_weights['enc_b1'] = tf.Variable(tf.zeros([self.n_hidden[1]], dtype = tf.float32))

        all_weights['enc_w2'] = tf.get_variable("enc_w2", shape=[self.kernel_size[2], self.kernel_size[2], self.n_hidden[1],self.n_hidden[2]],
            initializer=layers.xavier_initializer_conv2d(),regularizer = self.reg)
        all_weights['enc_b2'] = tf.Variable(tf.zeros([self.n_hidden[2]], dtype = tf.float32))        
        
        all_weights['Coef']   = tf.Variable(1.0e-4 * tf.ones([self.batch_size, self.batch_size],tf.float32), name = 'Coef')
        
        all_weights['dec_w0'] = tf.get_variable("dec_w0", shape=[self.kernel_size[2], self.kernel_size[2], self.n_hidden[1],self.n_hidden[2]],
            initializer=layers.xavier_initializer_conv2d(),regularizer = self.reg)
        all_weights['dec_b0'] = tf.Variable(tf.zeros([self.n_hidden[1]], dtype = tf.float32))

        all_weights['dec_w1'] = tf.get_variable("dec_w1", shape=[self.kernel_size[1], self.kernel_size[1], self.n_hidden[0],self.n_hidden[1]],
            initializer=layers.xavier_initializer_conv2d(),regularizer = self.reg)
        all_weights['dec_b1'] = tf.Variable(tf.zeros([self.n_hidden[0]], dtype = tf.float32))

        all_weights['dec_w2'] = tf.get_variable("dec_w2", shape=[self.kernel_size[0], self.kernel_size[0],1, self.n_hidden[0]],
            initializer=layers.xavier_initializer_conv2d(),regularizer = self.reg)
        all_weights['dec_b2'] = tf.Variable(tf.zeros([1], dtype = tf.float32))
        
        return all_weights
        
    # Building the encoder
    def encoder(self,x, weights):
        shapes = []
        # Encoder Hidden layer with relu activation #1
        shapes.append(x.get_shape().as_list())
        layer1 = tf.nn.bias_add(tf.nn.conv2d(x, weights['enc_w0'], strides=[1,2,2,1],padding='SAME'),weights['enc_b0'])
        layer1 = tf.nn.relu(layer1)
        shapes.append(layer1.get_shape().as_list())
        layer2 = tf.nn.bias_add(tf.nn.conv2d(layer1, weights['enc_w1'], strides=[1,2,2,1],padding='SAME'),weights['enc_b1'])
        layer2 = tf.nn.relu(layer2)
        shapes.append(layer2.get_shape().as_list())
        layer3 = tf.nn.bias_add(tf.nn.conv2d(layer2, weights['enc_w2'], strides=[1,2,2,1],padding='SAME'),weights['enc_b2'])
        layer3 = tf.nn.relu(layer3)
        return  layer3, shapes
    
    # Building the decoder
    def decoder(self,z, weights, shapes):
        # Encoder Hidden layer with relu activation #1
        shape_de1 = shapes[2]
        layer1 = tf.add(tf.nn.conv2d_transpose(z, weights['dec_w0'], tf.stack([tf.shape(self.x)[0],shape_de1[1],shape_de1[2],shape_de1[3]]),\
         strides=[1,2,2,1],padding='SAME'),weights['dec_b0'])
        layer1 = tf.nn.relu(layer1)
        shape_de2 = shapes[1]
        layer2 = tf.add(tf.nn.conv2d_transpose(layer1, weights['dec_w1'], tf.stack([tf.shape(self.x)[0],shape_de2[1],shape_de2[2],shape_de2[3]]),\
         strides=[1,2,2,1],padding='SAME'),weights['dec_b1'])
        layer2 = tf.nn.relu(layer2)
        shape_de3= shapes[0]
        layer3 = tf.add(tf.nn.conv2d_transpose(layer2, weights['dec_w2'], tf.stack([tf.shape(self.x)[0],shape_de3[1],shape_de3[2],shape_de3[3]]),\
         strides=[1,2,2,1],padding='SAME'),weights['dec_b2'])
        layer3 = tf.nn.relu(layer3)
        return layer3
    
    def partial_fit(self, X, lr): #  
        cost, summary, _, Coef = self.sess.run((self.reconst_cost, self.merged_summary_op, self.optimizer, self.Coef), feed_dict = {self.x: X, self.learning_rate: lr})#
        self.summary_writer.add_summary(summary, self.iter)
        self.iter = self.iter + 1
        return cost, Coef
    
    def initlization(self):
        self.sess.run(self.init)
    
    def reconstruct(self,X):
        return self.sess.run(self.x_r, feed_dict = {self.x:X})
    
    def transform(self, X):
        return self.sess.run(self.z, feed_dict = {self.x:X})
    
    def save_model(self):
        save_path = self.saver.save(self.sess,self.model_path)
        print ("model saved in file: %s" % save_path)

    def restore(self):
        self.saver.restore(self.sess, self.restore_path)
        print ("model restored")
        
def best_map(L1,L2):
    #L1 should be the groundtruth labels and L2 should be the clustering labels we got
    Label1 = np.unique(L1)
    nClass1 = len(Label1)
    Label2 = np.unique(L2)
    nClass2 = len(Label2)
    nClass = np.maximum(nClass1,nClass2)
    G = np.zeros((nClass,nClass))
    for i in range(nClass1):
        ind_cla1 = L1 == Label1[i]
        ind_cla1 = ind_cla1.astype(float)
        for j in range(nClass2):
            ind_cla2 = L2 == Label2[j]
            ind_cla2 = ind_cla2.astype(float)
            G[i,j] = np.sum(ind_cla2 * ind_cla1)
    m = Munkres()
    index = m.compute(-1*G.T)
    index = np.array(index)
    c = index[:,1]
    newL2 = np.zeros(L2.shape)
    for i in range(nClass2):
        newL2[L2 == Label2[i]] = Label1[c[i]]
    return newL2   

def thrC(C,ro):
    if ro < 1:
        N = C.shape[1]
        Cp = np.zeros((N,N))
        S = np.abs(np.sort(-np.abs(C),axis=0))
        Ind = np.argsort(-np.abs(C),axis=0)
        for i in range(N):
            cL1 = np.sum(S[:,i]).astype(float)
            stop = False
            csum = 0
            t = 0
            while(stop == False):
                csum = csum + S[t,i]
                if csum > ro*cL1:
                    stop = True
                    Cp[Ind[0:t+1,i],i] = C[Ind[0:t+1,i],i]
                t = t + 1
    else:
        Cp = C

    return Cp

def build_aff(C):
    N = C.shape[0]
    Cabs = np.abs(C)
    ind = np.argsort(-Cabs,0)
    for i in range(N):
        Cabs[:,i]= Cabs[:,i] / (Cabs[ind[0,i],i] + 1e-6)
    Cksym = Cabs + Cabs.T
    return Cksym

def post_proC(C, K, d, alpha):
    # C: coefficient matrix, K: number of clusters, d: dimension of each subspace
    C = 0.5*(C + C.T)
    r = d*K + 1
    U, S, _ = svds(C,r,v0 = np.ones(C.shape[0]))
    U = U[:,::-1]    
    S = np.sqrt(S[::-1])
    S = np.diag(S)    
    U = U.dot(S)    
    U = normalize(U, norm='l2', axis = 1)       
    Z = U.dot(U.T)
    Z = Z * (Z>0)    
    L = np.abs(Z ** alpha) 
    L = L/L.max()   
    L = 0.5 * (L + L.T)    
    spectral = cluster.SpectralClustering(n_clusters=K, eigen_solver='arpack', affinity='precomputed',assign_labels='discretize')
    spectral.fit(L)
    grp = spectral.fit_predict(L) + 1
    return grp, L

def err_rate(gt_s, s):
    c_x = best_map(gt_s,s)
    err_x = np.sum(gt_s[:] != c_x[:])
    missrate = err_x.astype(float) / (gt_s.shape[0])
    return missrate 

def build_laplacian(C):
    C = 0.5 * (np.abs(C) + np.abs(C.T))
    W = np.sum(C,axis=0)         
    W = np.diag(1.0/W)
    L = W.dot(C)    
    return L
 
        
def test_face(Img, Label, CAE, num_class):       
    
    alpha = max(0.4 - (num_class-1)/10 * 0.1, 0.1)
    print(alpha)

    acc_= []
    for i in range(0,39-num_class): 
        face_10_subjs = np.array(Img[64*i:64*(i+num_class),:])
        face_10_subjs = face_10_subjs.astype(float)        
        label_10_subjs = np.array(Label[64*i:64*(i+num_class)]) 
        label_10_subjs = label_10_subjs - label_10_subjs.min() + 1
        label_10_subjs = np.squeeze(label_10_subjs)    
        
            
        CAE.initlization()        
        CAE.restore() # restore from pre-trained model    
        
        max_step =  50 + num_class*25# 100+num_class*20
        display_step = max_step
        lr = 1.0e-3
        # fine-tune network
        epoch = 0
        while epoch < max_step:
            epoch = epoch + 1           
            cost, Coef = CAE.partial_fit(face_10_subjs, lr)#                                  
            if epoch % display_step == 0:
                print( "epoch: %.1d" % epoch, "cost: %.8f" % (cost/float(CAE.batch_size)))
                Coef = thrC(Coef,alpha)                                          
                y_x, _ = post_proC(Coef, label_10_subjs.max(), 10, 3.5)                  
                missrate_x = err_rate(label_10_subjs, y_x)                
                acc_x = 1 - missrate_x 
                print ("experiment: %d" % i, "our accuracy: %.4f" % acc_x)
        acc_.append(acc_x)    
    
    acc_ = np.array(acc_)
    m = np.mean(acc_)
    me = np.median(acc_)
    print("%d subjects:" % num_class)    
    print("Mean: %.4f%%" % ((1-m)*100))    
    print("Median: %.4f%%" % ((1-me)*100))
    print(acc_) 
    
    return (1-m), (1-me)  
    
    
# load face images and labels
data = sio.loadmat('DSC-rebuilt/Data/YaleBCrop025.mat')
img = data['Y']
I = []
Label = []
for i in range(img.shape[2]):
    for j in range(img.shape[1]):
        temp = np.reshape(img[:,j,i],[42,48])
        Label.append(i)
        I.append(temp)
I = np.array(I)
Label = np.array(Label[:])
Img = np.transpose(I,[0,2,1])
Img = np.expand_dims(Img[:],3)    

# face image clustering
n_input = [48,42]
kernel_size = [5,3,3]
n_hidden = [10,20,30]

all_subjects = [10, 15, 20, 25, 30, 35, 38]

avg = []
med = []

iter_loop = 0
while iter_loop < len(all_subjects):
    tic = time.time()
    num_class = all_subjects[iter_loop]
    batch_size = num_class * 64
    reg1 = 1.0
    reg2 = 1.0 * 10 ** (num_class / 10.0 - 3.0)           
    model_path = 'DSC-rebuilt/pretrain-model-EYaleB/model-102030-48x42-yaleb.ckpt' 
    restore_path = 'DSC-rebuilt/pretrain-model-EYaleB/model-102030-48x42-yaleb.ckpt' 
    logs_path = 'DSC-rebuilt/pretrain-model-EYaleB/logs' 
    tf.reset_default_graph()
    CAE = ConvAE(n_input=n_input, n_hidden=n_hidden, reg_constant1=reg1, re_constant2=reg2, \
                    kernel_size=kernel_size, batch_size=batch_size, model_path=model_path, restore_path=restore_path, logs_path=logs_path)

    avg_i, med_i = test_face(Img, Label, CAE, num_class)
    avg.append(avg_i)
    med.append(med_i)
    iter_loop = iter_loop + 1
    toc = time.time()
    print("excute time:"+str((toc-tic))+"s")
    
iter_loop = 0
while iter_loop < len(all_subjects):
    num_class = all_subjects[iter_loop]
    print ('%d subjects:' % num_class)
    print ('Mean: %.4f%%' % (avg[iter_loop]*100), 'Median: %.4f%%' % (med[iter_loop]*100))
    iter_loop = iter_loop + 1      

0.31
INFO:tensorflow:Restoring parameters from DSC-rebuilt/pretrain-model-EYaleB/model-102030-48x42-yaleb.ckpt


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


model restored
epoch: 300 cost: 56252.58125000
experiment: 0 our accuracy: 0.9844
INFO:tensorflow:Restoring parameters from DSC-rebuilt/pretrain-model-EYaleB/model-102030-48x42-yaleb.ckpt
model restored
epoch: 300 cost: 57363.46875000
experiment: 1 our accuracy: 0.9812
INFO:tensorflow:Restoring parameters from DSC-rebuilt/pretrain-model-EYaleB/model-102030-48x42-yaleb.ckpt
model restored
epoch: 300 cost: 56912.36875000
experiment: 2 our accuracy: 0.9703
INFO:tensorflow:Restoring parameters from DSC-rebuilt/pretrain-model-EYaleB/model-102030-48x42-yaleb.ckpt
model restored
epoch: 300 cost: 62084.02500000
experiment: 3 our accuracy: 0.9703
INFO:tensorflow:Restoring parameters from DSC-rebuilt/pretrain-model-EYaleB/model-102030-48x42-yaleb.ckpt
model restored
epoch: 300 cost: 64341.33125000
experiment: 4 our accuracy: 0.9844
INFO:tensorflow:Restoring parameters from DSC-rebuilt/pretrain-model-EYaleB/model-102030-48x42-yaleb.ckpt
model restored
epoch: 300 cost: 63818.71250000
experiment: 5

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


model restored
epoch: 425 cost: 47413.43750000
experiment: 0 our accuracy: 0.9812
INFO:tensorflow:Restoring parameters from DSC-rebuilt/pretrain-model-EYaleB/model-102030-48x42-yaleb.ckpt
model restored
epoch: 425 cost: 47672.53750000
experiment: 1 our accuracy: 0.9802
INFO:tensorflow:Restoring parameters from DSC-rebuilt/pretrain-model-EYaleB/model-102030-48x42-yaleb.ckpt
model restored
epoch: 425 cost: 47106.76250000
experiment: 2 our accuracy: 0.9812
INFO:tensorflow:Restoring parameters from DSC-rebuilt/pretrain-model-EYaleB/model-102030-48x42-yaleb.ckpt
model restored
epoch: 425 cost: 46445.34166667
experiment: 3 our accuracy: 0.9833
INFO:tensorflow:Restoring parameters from DSC-rebuilt/pretrain-model-EYaleB/model-102030-48x42-yaleb.ckpt
model restored
epoch: 425 cost: 45276.13750000
experiment: 4 our accuracy: 0.9833
INFO:tensorflow:Restoring parameters from DSC-rebuilt/pretrain-model-EYaleB/model-102030-48x42-yaleb.ckpt
model restored
epoch: 425 cost: 44603.37083333
experiment: 5

...

model restored
epoch: 800 cost: 38220.45833333
experiment: 0 our accuracy: 0.9839
INFO:tensorflow:Restoring parameters from DSC-rebuilt/pretrain-model-EYaleB/model-102030-48x42-yaleb.ckpt
model restored
epoch: 800 cost: 37744.60833333
experiment: 1 our accuracy: 0.9849
INFO:tensorflow:Restoring parameters from DSC-rebuilt/pretrain-model-EYaleB/model-102030-48x42-yaleb.ckpt
model restored
epoch: 800 cost: 37322.42083333
experiment: 2 our accuracy: 0.9812
INFO:tensorflow:Restoring parameters from DSC-rebuilt/pretrain-model-EYaleB/model-102030-48x42-yaleb.ckpt
model restored
epoch: 800 cost: 37010.05833333
experiment: 3 our accuracy: 0.9823
INFO:tensorflow:Restoring parameters from DSC-rebuilt/pretrain-model-EYaleB/model-102030-48x42-yaleb.ckpt
model restored
epoch: 800 cost: 36772.18333333
experiment: 4 our accuracy: 0.9776
INFO:tensorflow:Restoring parameters from DSC-rebuilt/pretrain-model-EYaleB/model-102030-48x42-yaleb.ckpt
model restored
experiment: 5 our accuracy: 0.9531
INFO:tenso

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


model restored
epoch: 925 cost: 58330.80000000
experiment: 0 our accuracy: 0.9763
INFO:tensorflow:Restoring parameters from DSC-rebuilt/pretrain-model-EYaleB/model-102030-48x42-yaleb.ckpt
model restored
epoch: 925 cost: 57896.50357143
experiment: 1 our accuracy: 0.9737
INFO:tensorflow:Restoring parameters from DSC-rebuilt/pretrain-model-EYaleB/model-102030-48x42-yaleb.ckpt
model restored
epoch: 925 cost: 57372.95714286
experiment: 2 our accuracy: 0.9754
INFO:tensorflow:Restoring parameters from DSC-rebuilt/pretrain-model-EYaleB/model-102030-48x42-yaleb.ckpt
model restored
epoch: 925 cost: 57163.47857143
experiment: 3 our accuracy: 0.9763
35 subjects:
Mean: 2.4554%
Median: 2.4107%
[0.97633929 0.97366071 0.97544643 0.97633929]
excute time:240.8405089378357s
0.1
INFO:tensorflow:Restoring parameters from DSC-rebuilt/pretrain-model-EYaleB/model-102030-48x42-yaleb.ckpt


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


model restored
epoch: 1000 cost: 86401.82894737
experiment: 0 our accuracy: 0.9729
38 subjects:
Mean: 2.7138%
Median: 2.7138%
[0.97286184]
excute time:75.25233769416809s
10 subjects:
Mean: 1.5140% Median: 1.4062%
15 subjects:
Mean: 1.5234% Median: 1.6146%
20 subjects:
Mean: 1.4679% Median: 1.4844%
25 subjects:
Mean: 1.6518% Median: 1.6563%
30 subjects:
Mean: 2.9861% Median: 2.2396%
35 subjects:
Mean: 2.4554% Median: 2.4107%
38 subjects:
Mean: 2.7138% Median: 2.7138%


In [5]:
!git commit -a

fatal: not a git repository (or any of the parent directories): .git
